# Start here

In [1]:
import pandas as pd
import numpy as np
import random


exp_dat = pd.read_csv('trial_file_626708_20220524_092356_UTC.csv', header=0)
exp_dat.loc[exp_dat.condition1 == 'Distractor', 'type'] = 'distractor'
for dtarget, vnum in zip(exp_dat.loc[exp_dat.type == 'distractor'].target.unique(), range(1,13)):
    exp_dat.loc[exp_dat.target == dtarget, 'condition3'] = str(vnum)

In [2]:
trial_types = ['test', 'distractor']
vig_numbers = list(range(1, 13))
bel_types =  ['TB', 'FB', 'IG']
ascrip_types = ['Knows','Thinks']

stimuli = {trial_type:{vnum:{
                             'belief_manip':{b:None for b in bel_types},
                             'ascription':{a:None for a in ascrip_types}
                             }
                       for vnum in vig_numbers}
           for trial_type in trial_types }

In [3]:

for trial_type in trial_types:
    for vnum in vig_numbers:
        for bel in bel_types:
            for ascrip in ascrip_types:
                if trial_type == 'distractor':
                    condition = exp_dat.loc[(exp_dat.type == trial_type) &
                                            (exp_dat.condition3 == str(vnum)) &
                                            (exp_dat.subjectGroup == str(1))]
                else:
                    condition = exp_dat.loc[(exp_dat.type == trial_type) &
                                            (exp_dat.condition1 == ascrip) &
                                            (exp_dat.condition2 == bel) &
                                            (exp_dat.condition3 == str(vnum))]

                stimuli[trial_type][vnum]['belief_manip'][bel] = exp_dat.loc[condition.index.values[0] - 5: condition.index.values[0] - 2, 'title'].values.tolist()
                stimuli[trial_type][vnum]['ascription'][ascrip] = {'target':condition.target.values[0], 'crrct_answr':condition.key.values[0]}

# Add practice

In [4]:
# Adding Practice
p1 = ["Kevin is out grocery shopping at the local market. His wife told him to get sourdough bread and pasta for dinner that evening.",
      "At the market, Kevin picks up some pasta, but he cannot find any sourdough bread anywhere. He looks up and down every aisle and then finally asks someone who works at the market.",
      "It turns out that the bakery at the market no longer makes sourdough bread because it was never very popular. ",
      "Kevin goes home without bread of any kind. His wife is slightly annoyed."
      ]
p1_target = "Kevin asked someone at the store if they sold bread."
p2 = ["Sara's younger sister is late for dance class. Sara has a scooter that she could lend her sister for the afternoon.",
      "The scooter is brand-new and will get Sara's sister to her dance class right on time. But Sara wants to use her scooter to get to her friend's house for dinner later on.",
      "Sara doesn't think that her sister's dance class will be over in time for her to also be able to use the scooter to get to dinner.",
      "Sara doesn't lend the scooter to her sister. Her sister walks to dance class and is late."
      ]
p2_target = "Sara wanted to take her scooter to the park."
stimuli['practice'] = {1:{'belief_manip':{bel:p1 for bel in bel_types},
                       'ascription':{a:{'target':p1_target, 'crrct_answer': 'j'} for a in ascrip_types}
                    },
                       2:{'belief_manip':{bel:p2 for bel in bel_types},
                       'ascription':{a:{'target':p2_target, 'crrct_answer': 'f'} for a in ascrip_types}
                    }}



# save stim json

In [5]:
import json

with open('KB_stim.json', 'w') as outfile:
    outfile.write(json.dumps(stimuli))
#stimuli.to_json('KbeforeB_stim.js', orient='records')


# Randomizing Trials for a Single Participant

In [1]:
import json
import pandas as pd
import numpy as np
import random


'''
def randomize_trials():
    trial_types = ['test', 'distractor']
    vig_numbers = list(range(1, 13))
    bel_types = ['TB', 'FB', 'IG']
    ascrip_types = ['Knows', 'Thinks']
    # assign scenarios and shuffle
    s1 = pd.DataFrame({
        'trial_num': None,
        'prompt': None,
        'correct': None,
        'trial_type': 'test',
        'scenario': vig_numbers,
        'belief': None,
        'ascription': None,
        'target': None,
        'crrct_answer': None,
        'target_onset': None,
        'response_onset': None,
        'response_key': None,
    }).sample(frac=1)
    # balance balance acription types and shuffly
    s1['ascription'] = np.repeat(ascrip_types, len(vig_numbers) / len(ascrip_types))
    s1 = s1.sample(frac=1)
    # balance belief types and shuffl
    s1['belief'] = np.repeat(bel_types, len(vig_numbers) / len(bel_types))
    s2 = s1.copy()
    s2['trial_type'] = 'distractor'
    s3 = pd.concat([s1, s2], ignore_index=True).sample(frac=1).reset_index(drop=True)
    s3['trial_num'] = range(1, len(s3) + 1)
    # add practice
    # Adding Practice
    p1 = [
        "Kevin is out grocery shopping at the local market. His wife told him to get sourdough bread and pasta for dinner that evening.",
        "At the market, Kevin picks up some pasta, but he cannot find any sourdough bread anywhere. He looks up and down every aisle and then finally asks someone who works at the market.",
        "It turns out that the bakery at the market no longer makes sourdough bread because it was never very popular. ",
        "Kevin goes home without bread of any kind. His wife is slightly annoyed."
        ]
    p1_target = "Kevin asked someone at the store if they sold bread."
    p2 = [
        "Sara's younger sister is late for dance class. Sara has a scooter that she could lend her sister for the afternoon.",
        "The scooter is brand-new and will get Sara's sister to her dance class right on time. But Sara wants to use her scooter to get to her friend's house for dinner later on.",
        "Sara doesn't think that her sister's dance class will be over in time for her to also be able to use the scooter to get to dinner.",
        "Sara doesn't lend the scooter to her sister. Her sister walks to dance class and is late."
        ]
    p2_target = "Sara wanted to take her scooter to the park."
    n_practice = 8
    s3a = pd.DataFrame({x: None for x in s3.columns.tolist()}, index=range(n_practice))
    s3a.trial_type = ['tf_practice', 'tf_practice', 'tf_practice', 'tf_practice', 'tf_practice', 'tf_practice',
                      'practice', 'practice']
    s3a.trial_num = [999] * n_practice

    tf_prac = ["False"] * 3 + ["True"] * 3
    random.shuffle(tf_prac)
    ca = ['f' if tf_p == 'False' else 'j' for tf_p in tf_prac]
    [ca.append(x) for x in ['j', 'f']]
    [tf_prac.append(prac_prompt) for prac_prompt in [p1, p2]]
    pr_targ = [None] * 6
    [pr_targ.append(p_target) for p_target in [p1_target, p2_target]]

    s3a.prompt = tf_prac
    s3a.target = pr_targ
    s3a.crrct_answer = ca
    s4 = s3a.append(s3, ignore_index=True)
    xx = s4.loc[s4.trial_type == 'test'].sample(frac=1)
    fel = pd.DataFrame({'b2_trial_num': list(range(1, 13)),
                        'b1_trial_num': xx.trial_num.values,
                        'fel_scenario': xx.scenario.values,
                        'fel_belief_type': xx.belief.values,
                        'fel_ascription_type': xx.ascription.values,  # "knows" or "thinks"
                        'felicity_rating': None})

    return s4, fel
'''

'\ndef randomize_trials():\n    trial_types = [\'test\', \'distractor\']\n    vig_numbers = list(range(1, 13))\n    bel_types = [\'TB\', \'FB\', \'IG\']\n    ascrip_types = [\'Knows\', \'Thinks\']\n    # assign scenarios and shuffle\n    s1 = pd.DataFrame({\n        \'trial_num\': None,\n        \'prompt\': None,\n        \'correct\': None,\n        \'trial_type\': \'test\',\n        \'scenario\': vig_numbers,\n        \'belief\': None,\n        \'ascription\': None,\n        \'target\': None,\n        \'crrct_answer\': None,\n        \'target_onset\': None,\n        \'response_onset\': None,\n        \'response_key\': None,\n    }).sample(frac=1)\n    # balance balance acription types and shuffly\n    s1[\'ascription\'] = np.repeat(ascrip_types, len(vig_numbers) / len(ascrip_types))\n    s1 = s1.sample(frac=1)\n    # balance belief types and shuffl\n    s1[\'belief\'] = np.repeat(bel_types, len(vig_numbers) / len(bel_types))\n    s2 = s1.copy()\n    s2[\'trial_type\'] = \'distractor\'

In [2]:
sub_trials, felicity = randomize_trials()


In [3]:
practice  = sub_trials.loc[sub_trials.trial_num == 999]
sub_trials = sub_trials.loc[~sub_trials.trial_num == 999].reset_index(drop=True)

Re-do it idiot

In [2]:
# practice
def make_practice():
    n_tf_pract = 8
    n_story_pract = 2
    p1 = [
        "Kevin is out grocery shopping at the local market. His wife told him to get sourdough bread and pasta for dinner that evening.",
        "At the market, Kevin picks up some pasta, but he cannot find any sourdough bread anywhere. He looks up and down every aisle and then finally asks someone who works at the market.",
        "It turns out that the bakery at the market no longer makes sourdough bread because it was never very popular. ",
        "Kevin goes home without bread of any kind. His wife is slightly annoyed."
        ]
    p1_target = "Kevin asked someone at the store if they sold bread."
    p2 = [
        "Sara's younger sister is late for dance class. Sara has a scooter that she could lend her sister for the afternoon.",
        "The scooter is brand-new and will get Sara's sister to her dance class right on time. But Sara wants to use her scooter to get to her friend's house for dinner later on.",
        "Sara doesn't think that her sister's dance class will be over in time for her to also be able to use the scooter to get to dinner.",
        "Sara doesn't lend the scooter to her sister. Her sister walks to dance class and is late."
        ]
    p2_target = "Sara wanted to take her scooter to the park."
    tf_prac = ["False"] * int(n_tf_pract/2) + ["True"] * int(n_tf_pract/2)
    random.shuffle(tf_prac)
    ca = ['f' if tf_p == 'False' else 'j' for tf_p in tf_prac]
    [ca.append(x) for x in ['j', 'f']]
    pract = {'trial_num': list(range(1, sum([n_tf_pract,n_story_pract])+1)),
             'trial_type':['tf_practic'] * n_tf_pract + ['story_practice']* n_story_pract,
             'prompt': tf_prac + [p1, p2],
             'correct': None,
             'target': [None]*n_tf_pract + [p1_target, p2_target],
             'correct_answer': ca,
             'response_key':None,
             'target_onset': None,
             'response_onset': None,
             }
    return pd.DataFrame(pract)

In [3]:
# Trials
def make_trials():
    vig_numbers = list(range(1, 13))
    bel_types = ['TB', 'FB', 'IG']
    ascrip_types = ['Knows', 'Thinks']
    # assign scenarios and shuffle
    s1 = pd.DataFrame({
        'trial_num': None,
        'correct': None,
        'trial_type': 'test',
        'scenario': vig_numbers,
        'belief': None,
        'ascription': None,
        'crrct_answer': None,
        'target_onset': None,
        'response_onset': None,
        'response_key': None,
    }).sample(frac=1)
    # balance ascription types and shuffle
    s1['ascription'] = np.repeat(ascrip_types, len(vig_numbers) / len(ascrip_types))
    s1 = s1.sample(frac=1)
    # balance belief types and shuffle
    s1['belief'] = np.repeat(bel_types, len(vig_numbers) / len(bel_types))
    # add distractors
    s2 = s1.copy()
    s2['trial_type'] = 'distractor'
    s3 = pd.concat([s1, s2], ignore_index=True).sample(frac=1).reset_index(drop=True)
    s3['trial_num'] = range(1, len(s3) + 1)
    # add correct answers
    #json_fp = 'KnowledgeBelief/static/stim_data/KB_stim.json'
    json_fp = 'KB_stim.json'
    with open(json_fp, 'r') as j:
        stim = json.loads(j.read())
    for i, trl in s3.iterrows():
        s3.loc[s3.trial_num == trl.trial_num, 'crrct_answer'] = stim[trl.trial_type][str(trl.scenario)]['ascription'][trl.ascription]['crrct_answr']
    return s3

In [4]:
# Felicity
def make_felicity(trls):
    xx = trls.loc[trls.trial_type == 'test'].sample(frac=1)
    fel = pd.DataFrame({'b2_trial_num': list(range(1, 13)),
                        'b1_trial_num': xx.trial_num.values,
                        'fel_scenario': xx.scenario.values,
                        'fel_belief_type': xx.belief.values,
                        'fel_ascription_type': xx.ascription.values,  # "knows" or "thinks"
                        'felicity_rating': None})
    return fel


In [5]:
def randomize_trials():
    prac = make_practice()
    trls = make_trials()
    fels = make_felicity(trls)
    return prac, trls, fels

In [6]:
s_practice, s_trials, s_fels = randomize_trials()

# Messages

In [ ]:
exp_states  = ['TRIAL_PRACTICE', 'TF_TRIAL', 'FELICITY_PRACTICE']
m = [1, 2, 'next']
msgs = {e_state:{var:None for var in m} for e_state in exp_states}

msgs[exp_states[0]][1] = "Okay, one more practice example"
msgs[exp_states[0]][2] = "Press the space bar to continue, then place your fingers on the [f] and [j] keys.... "
msgs[exp_states[0]]['next'] = "/story"
msgs[exp_states[1]][1] = "Okay, get ready for story #" + request.args.get('trial') + "."
msgs[exp_states[1]][2] = "Press the space bar to continue, then place your fingers on the [f] and [j] keys.... "
msgs[exp_states[1]]['next'] = "/story"
msgs[exp_states[1]][1] = "Great Job!"
msgs[exp_states[1]][2] = "Press the space bar to continue.... "
msgs[exp_states[1]]['next'] = "/felicity_instr"